<a href="https://colab.research.google.com/github/AgustinCocciardi/TP1_P2_ProgConcu/blob/main/TP1_P2_Threads_C%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Utilizo el comando writefile para crear un archivo llamado tp2.cpp. Necesitaré ese archivo para crear un ejecutable.

In [ ]:
%%writefile tp2.cpp
#include <iostream>
#include <string>
#include <thread>
#include <cstdlib>
#include <ctime>
#include <iomanip>
#include <vector>
#include <ctype.h>
#include <cstring>

#define PARAMETROS_INCORRECTOS 0
#define N_FUERA_DE_RANGO PARAMETROS_INCORRECTOS
#define PARAMETRO_NO_ENTERO PARAMETROS_INCORRECTOS
#define ESPACIO_CARACTERES 6
#define INICIO_ITERADOR 0
#define PARAMETROS_CORRECTOS 2
#define RANGO_MINIMO 5
#define RANGO_MAXIMO 20
#define VALOR_ALEATORIO_MAXIMO 65
#define VALOR_ALEATORIO_NEGATIVO 32
#define EXITO 1
#define INICIALIZADOR 0

struct MatricesHilos
{
  int** matrizA;
  int** matrizB;
  int** matrizCS;
  int** matrizCH;
  int tamanioMatriz;
  int cantidadHilos;
};

bool cantidadCorrectaDeParametros(int parametrosRecibidos, int parametrosEsperados);
bool parametroNEsEntero(char* parametro);
bool rangoValidoN(int valor, int valorMinimo, int valorMaximo);
int** crearMatriz(int tamanioMatriz);
void mostrarMatriz(int **matriz,int tamanioMatriz,const char* nombre);
MatricesHilos llenarEstructura(int tamanio);
void llenarMatricesAleatorio();
void multiplicarMatricesSecuencial();
void multiplicarMatricesConHilos(int cantidadHilos);
void mostrarMatrices();
void funcionEjecutadaPorHilo(int numeroHilo);
void compararMatricesResultado();

MatricesHilos matrizHilos;

int main(int argc, char* argv[])
{
  if(!cantidadCorrectaDeParametros(argc,PARAMETROS_CORRECTOS))
  {
    std::cout<<"No ingreso la cantidad correcta de parametros. Solamente debe ingresar un entero positivo entre 5 y 20"<<std::endl;
    return PARAMETROS_INCORRECTOS;
  }
  char* parametroN = argv[1];
  if (!parametroNEsEntero(parametroN))
  {
    std::cout<<"El parametro que usted ingreso no es un entero positivo entre 5 y 20"<<std::endl;
    return PARAMETRO_NO_ENTERO;
  }
  int tamanioMatrices = atoi(argv[1]);
  if (!rangoValidoN(tamanioMatrices,RANGO_MINIMO,RANGO_MAXIMO))
  {
    std::cout<<"El rango de N debe estar entre 5 y 20"<<std::endl;
    return N_FUERA_DE_RANGO;
  }
  
  matrizHilos = llenarEstructura(tamanioMatrices);
  llenarMatricesAleatorio();
  multiplicarMatricesSecuencial();
  multiplicarMatricesConHilos(matrizHilos.cantidadHilos);
  mostrarMatrices();
  compararMatricesResultado();
  return EXITO;
}

bool cantidadCorrectaDeParametros(int parametrosRecibidos, int parametrosEsperados)
{
  return parametrosRecibidos == parametrosEsperados;
}

bool parametroNEsEntero(char* parametro)
{
  for (int i = INICIO_ITERADOR; i < strlen(parametro); i++)
  {
    if (!isdigit(parametro[i]))
    {
      return false;
    }
  }
  return true;
}

bool rangoValidoN(int valor, int valorMinimo, int valorMaximo)
{
  return valor >= valorMinimo && valor <= valorMaximo;
}

int** crearMatriz(int tamanioMatriz)
{
  int** matriz = new int*[tamanioMatriz];
  for (int i = INICIO_ITERADOR; i < tamanioMatriz; i++)
  {
    matriz[i] = new int[tamanioMatriz];
    for (int j = INICIO_ITERADOR; j < tamanioMatriz; j++)
    {
      matriz[i][j] = INICIALIZADOR;
    }
  }
  return matriz;
}

void mostrarMatriz(int **matriz,int tamanioMatriz,const char* nombre)
{
  std::cout<<nombre<<std::endl;
  for (int i = INICIO_ITERADOR; i < tamanioMatriz; i++)
  {
    for (int j = INICIO_ITERADOR; j < tamanioMatriz; j++)
    {
      std::cout << std::setw(ESPACIO_CARACTERES) << matriz[i][j];
    }
    std::cout << std::endl;
  }
  std::cout << std::endl;
}

MatricesHilos llenarEstructura(int tamanio)
{
  MatricesHilos matrizHilo;
  matrizHilo.matrizA = crearMatriz(tamanio);
  matrizHilo.matrizB = crearMatriz(tamanio);
  matrizHilo.matrizCS = crearMatriz(tamanio);
  matrizHilo.matrizCH = crearMatriz(tamanio);    
  matrizHilo.tamanioMatriz = matrizHilo.cantidadHilos = tamanio;
  return matrizHilo;
}

void llenarMatricesAleatorio()
{
  srand(time(NULL));
  for (int i = INICIO_ITERADOR; i < matrizHilos.tamanioMatriz; i++)
  {
    for (int j = INICIO_ITERADOR; j < matrizHilos.tamanioMatriz; j++)
    {
      matrizHilos.matrizA[i][j] = rand() % VALOR_ALEATORIO_MAXIMO - VALOR_ALEATORIO_NEGATIVO;
      matrizHilos.matrizB[i][j] = rand() % VALOR_ALEATORIO_MAXIMO - VALOR_ALEATORIO_NEGATIVO;
    }
  }    
}

void multiplicarMatricesSecuencial()
{
  for (int i = INICIO_ITERADOR; i < matrizHilos.tamanioMatriz; i++)
  {
    for (int j = INICIO_ITERADOR; j < matrizHilos.tamanioMatriz; j++)
    {
      matrizHilos.matrizCS[i][j] = INICIALIZADOR;
      for (int k = INICIO_ITERADOR; k < matrizHilos.tamanioMatriz; k++)
      {
        matrizHilos.matrizCS[i][j] += matrizHilos.matrizA[i][k] * matrizHilos.matrizB[k][j];
      }
    }
  }    
}

void multiplicarMatricesConHilos(int cantidadHilos)
{
  std::vector<std::thread> hilos;
  for (int i = INICIO_ITERADOR; i < cantidadHilos; i++)
  {
    hilos.emplace_back(funcionEjecutadaPorHilo,i);
  }
  for (auto& hilo : hilos) 
  {
    hilo.join();
  }
}

void mostrarMatrices()
{
  mostrarMatriz(matrizHilos.matrizA,matrizHilos.tamanioMatriz,"Matriz A");
  mostrarMatriz(matrizHilos.matrizB,matrizHilos.tamanioMatriz,"Matriz B");
  mostrarMatriz(matrizHilos.matrizCS,matrizHilos.tamanioMatriz,"Matriz CS");
  mostrarMatriz(matrizHilos.matrizCH,matrizHilos.tamanioMatriz,"Matriz CH");
}

void funcionEjecutadaPorHilo(int numeroHilo)
{
  for (int j = INICIO_ITERADOR; j < matrizHilos.tamanioMatriz; j++)
  {
    for (int k = INICIO_ITERADOR; k < matrizHilos.tamanioMatriz; k++)
    {
      matrizHilos.matrizCH[numeroHilo][j] += matrizHilos.matrizA[numeroHilo][k] * matrizHilos.matrizB[k][j];
    }
  }
}

void compararMatricesResultado()
{
  bool sonIguales = true;
  for (int i = INICIO_ITERADOR; i < matrizHilos.tamanioMatriz; i++)
  {
    for (int j = INICIO_ITERADOR; j < matrizHilos.tamanioMatriz; j++)
    {
      if (matrizHilos.matrizCS[i][j] != matrizHilos.matrizCH[i][j])
      {
        sonIguales = false;
        break;
      }
    }
  }
  if (sonIguales)
    std::cout<<"El producto matricial coincide"<<std::endl;
  else
    std::cout<<"El producto matricial no coincide"<<std::endl;
}

Writing tp2.cpp


Listo los archivos para asegurarme que existe el nuevo archivo que cree (tp2.cpp).

In [ ]:
!ls -la

total 24
drwxr-xr-x 1 root root 4096 Apr 18 21:24 .
drwxr-xr-x 1 root root 4096 Apr 18 21:24 ..
drwxr-xr-x 4 root root 4096 Apr 14 13:34 .config
drwxr-xr-x 1 root root 4096 Apr 14 13:35 sample_data
-rw-r--r-- 1 root root 5943 Apr 18 21:24 tp2.cpp


Compilo el archivo tp2.cpp para obtener un ejecutable. En el codigo no utilizo la biblioteca pthread.h ni ninguna otra similar. Sin embargo, es necesario incluirla en el comando para crear el ejecutable. De lo contrario, arroja un error. 

Puede ver más detalles del error que arrojaría si no incluye el ptrhead en [este sitio](https://itslinuxfoss.com/undefined-reference-pthread-create/#:~:text=The%20error%20%E2%80%9Cundefined%20reference%20to,line%20while%20compiling%20the%20code.)

In [ ]:
!g++ -std=c++11 -pthread -o tp2 tp2.cpp

Listo los archivos de nuevo para asegurarme que poseo el ejecutable. Debería salir como tp2 sin ninguna clase de extensión.

In [ ]:
!ls -la

total 80
drwxr-xr-x 1 root root  4096 Apr 18 21:24 .
drwxr-xr-x 1 root root  4096 Apr 18 21:24 ..
drwxr-xr-x 4 root root  4096 Apr 14 13:34 .config
drwxr-xr-x 1 root root  4096 Apr 14 13:35 sample_data
-rwxr-xr-x 1 root root 55304 Apr 18 21:24 tp2
-rw-r--r-- 1 root root  5943 Apr 18 21:24 tp2.cpp


Utilizo el comando nohup para ejecutar el proceso en segundo plano, de manera que no me ocupe la consola. Es necesario añadir el caracter '&' al final.

Para ejecutar el proceso, tengo que poner ./tp2 N donde N es un número comprendido en el rango de 5 a 20. 

El comando 1>salidaC va a redirigir la salida estandar de la pantalla (stdout) a un archivo llamado salidaC. 

El comando 2>/dev/null envía la salida de error estandar (stderr) a dev/null

In [ ]:
!nohup ./tp2 5 1>salidaC 2>/dev/null & 

Reviso el archivo de salida. Debo mostrar la matriz A, la matriz B, la matriz CS (resultado de realizar el producto matricial de forma secuencial), la matriz CH (resultado de realizar el producto matricial utilizando hilos), y por último una línea de texto que me indique si las matrices resultantes son iguales o diferentes. 

In [ ]:
!cat salidaC

Matriz A
    -5    18    20    21   -19
    23    16   -23     0    32
   -14    22    -1    -1    -1
   -10    11    21     8     0
    28    17    -2   -20     8

Matriz B
    16    27    -1    32   -17
    26   -25     1    12    11
    21    25    -6   -22    26
    27   -23    -1    11    22
    -9    11    28    32   -19

Matriz CS
  1546  -777  -650  -761  1626
    13    -2  1027  2458 -1421
   309  -941    15  -205   451
   783  -204  -113  -562  1013
   236   829   245  1180  -933

Matriz CH
  1546  -777  -650  -761  1626
    13    -2  1027  2458 -1421
   309  -941    15  -205   451
   783  -204  -113  -562  1013
   236   829   245  1180  -933

El producto matricial coincide


Vuelvo a ejecutar el proceso. Esta vez pasándole 2 parámetros. Redirigiré la salida al archivo salidaExcesoParametros.

In [ ]:
!nohup ./tp2 5 2 1>salidaExcesoParametros 2>/dev/null & 

Muestro el archivo salidaExcesoParametros. Debe informarme que solo debo ingresar un el valor de N como parámetro. 

In [ ]:
!cat salidaExcesoParametros

No ingreso la cantidad correcta de parametros. Solamente debe ingresar un entero positivo entre 5 y 20


Vuelvo a ejecutar el proceso. Esta vez pasándole 0 parámetros. Redirigiré la salida al archivo salidaSinParametros.

In [ ]:
!nohup ./tp2 1>salidaSinParametros 2>/dev/null & 

Muestro el archivo salidaSinParametros. Debe informarme que solo debo ingresar un el valor de N como parámetro. 

In [ ]:
!cat salidaSinParametros

No ingreso la cantidad correcta de parametros. Solamente debe ingresar un entero positivo entre 5 y 20


Vuelvo a ejecutar el proceso. Esta vez pasándole un valor mayor a 20. Redirigiré la salida al archivo salidaRangoSuperiorExcedido.

In [ ]:
!nohup ./tp2 25 1>salidaRangoSuperiorExcedido 2>/dev/null & 

Muestro el archivo salidaRangoSuperiorExcedido. Debe informarme que debo ingresar un número entre 5 y 20 como parámetro. 

In [ ]:
!cat salidaRangoSuperiorExcedido

El rango de N debe estar entre 5 y 20


Vuelvo a ejecutar el proceso. Esta vez pasándole un valor menor a 5. Redirigiré la salida al archivo salidaRangoInferiorExcedido.

In [ ]:
!nohup ./tp2 3 1>salidaRangoInferiorExcedido 2>/dev/null & 

Muestro el archivo salidaRangoInferiorExcedido. Debe informarme que debo ingresar un número entre 5 y 20 como parámetro. 

In [ ]:
!cat salidaRangoInferiorExcedido

El rango de N debe estar entre 5 y 20


Vuelvo a ejecutar el proceso. Esta vez pasandole un numero no entero. Redirigiré la salida al archivo salidaNumeroNoEntero.

In [ ]:
!nohup ./tp2 3.1 1>salidaNumeroNoEntero 2>/dev/null & 

Muestro el archivo salidaNumeroNoEntero. Debe informarme que no ingresé un número entero positivo. 

In [ ]:
!cat salidaNumeroNoEntero

El parametro que usted ingreso no es un entero positivo entre 5 y 20
